In [2]:
%reload_ext autoreload
%autoreload 2

import torch
from transformers import CLIPModel, AutoProcessor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# processor = ViTImageProcessor.from_pretrained('google/vit-large-patch16-224')
# model = ViTForImageClassification.from_pretrained('google/vit-large-patch16-224').to(device)
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
# processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
raw_processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
# processor =  lambda images: raw_processor(images=images, return_tensors="pt", padding=False, do_resize=False, do_center_crop=False)["pixel_values"]
processor =  lambda images: raw_processor(images=images, return_tensors="pt")["pixel_values"]
text_processor =  lambda text: raw_processor(text=[text], return_tensors="pt", padding=True, truncation=True)
img_processor =  lambda images: raw_processor(images=images, return_tensors="pt")["pixel_values"]
model = model.eval()

In [3]:
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from PIL import Image

# can be any image
image_path = "crepe_data/VG_100K_2/1.jpg"
image = Image.open(image_path)
image = image.convert('RGB')

plt.imshow(image)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'crepe_data/VG_100K_2/5.jpg'

# How to embed an image with clip

In [8]:
def get_text_output_from_image(imgs, model):
    # inputs = processor(imgs, return_tensors="pt").to("cuda")

    with torch.no_grad():
        # Select the CLS token embedding from the last hidden layer
        # return model(pixel_values=imgs).last_hidden_state[:, 0, :]
        return model.get_image_features(pixel_values=imgs, output_hidden_states=True)

In [10]:
def _to_device(x, device):
    if isinstance(x, torch.Tensor):
        return x.to(device)
    elif isinstance(x, list):
        return [_to_device(xi, device) for xi in x]

patches = processor([image])
x = _to_device(patches, device)
image_emb = get_text_output_from_image(x, model)
print(image_emb.shape)

torch.Size([1, 768])


# How to embed text with clip

In [11]:
query="the door has a frame around it," # and there is a second window on the building."
inputs = text_processor(query)
inputs = {key: val.to(device) for key, val in inputs.items()}
text_features = model.get_text_features(**inputs)
print(text_features.shape)

torch.Size([1, 768])
